# Lab 1-2: Hello World Pipeline

## 📋 실습 개요

| 항목 | 내용 |
|------|------|
| **소요시간** | 40분 |
| **난이도** | ⭐⭐ |
| **목표** | Kubeflow Pipelines로 첫 번째 ML 워크플로우 작성 및 실행 |

## 🎯 학습 목표

이 실습을 통해 다음을 학습합니다:
- **KFP SDK v2** 기본 개념 이해
- **Component** 정의 방법
- **Pipeline** 구성 방법
- **파이프라인 컴파일** (Python → YAML)
- **Kubeflow UI**를 통한 파이프라인 업로드 및 실행

---

## 🏗️ 파이프라인 구조

```
입력 파라미터
├─ a = 10
├─ b = 20
└─ factor = 3

실행 흐름 (DAG)
┌─────────────┐
│     add     │  ← a, b를 입력받아 합계 계산
│  (a + b)    │
└──────┬──────┘
       │ sum = 30
       ▼
┌─────────────┐
│  multiply   │  ← sum과 factor를 곱함
│ (sum × f)   │
└──────┬──────┘
       │ product = 90
       ▼
┌─────────────┐
│print_result │  ← 최종 결과 출력
│   (출력)    │
└─────────────┘
```

**계산 과정:**
1. `add(10, 20)` → 30
2. `multiply(30, 3)` → 90
3. `print_result(90)` → "Final Result: 90"

---

## Cell 0: 패키지 설치 및 임포트

In [ ]:
# KFP SDK 설치 (필요한 경우)
!pip install kfp>=2.0.0 -q

In [ ]:
# 패키지 임포트
from kfp import dsl
from kfp import compiler

# 버전 확인
import kfp
print(f"KFP Version: {kfp.__version__}")

---

## Cell 1: Component 정의 - add

**Component**는 파이프라인의 한 단계를 수행하는 독립적인 함수입니다.

**특징:**
- Python 함수로 정의
- `@dsl.component` 데코레이터 사용
- 입력/출력 타입 힌트 필수
- 각 컴포넌트는 별도의 컨테이너에서 실행

In [ ]:
@dsl.component(base_image='python:3.11')
def add(a: int, b: int) -> int:
    result = a + b
    print(f"Add: {a} + {b} = {result}")
    return result

print("✅ add Component 정의 완료!")

---

## Cell 2: Component 정의 - multiply

In [ ]:
@dsl.component(base_image='python:3.11')
def multiply(x: int, factor: int = 2) -> int:
    result = x * factor
    print(f"Multiply: {x} * {factor} = {result}")
    return result

print("✅ multiply Component 정의 완료!")

---

## Cell 3: Component 정의 - print_result

In [ ]:
@dsl.component(base_image='python:3.11')
def print_result(value: int):
    print("=" * 50)
    print(f"Final Result: {value}")
    print("=" * 50)

print("✅ print_result Component 정의 완료!")

---

## Cell 4: Pipeline 정의

**Pipeline**은 여러 Component를 연결한 워크플로우입니다.

**특징:**
- `@dsl.pipeline` 데코레이터 사용
- Component들의 실행 순서 정의
- Component 간 데이터 전달 (`.output` 사용)
- 파라미터로 실행 시 값 변경 가능

In [ ]:
@dsl.pipeline(
    name='Hello World Pipeline',
    description='Simple addition and multiplication pipeline'
)
def hello_pipeline(
    a: int = 3,
    b: int = 5,
    factor: int = 2
):
    # Step 1: a + b 계산
    add_task = add(a=a, b=b)
    
    # Step 2: (a + b) * factor 계산
    # add_task.output을 multiply의 x 파라미터로 전달
    multiply_task = multiply(
        x=add_task.output,
        factor=factor
    )
    
    # Step 3: 최종 결과 출력
    # multiply_task.output을 print_result의 value 파라미터로 전달
    print_result(value=multiply_task.output)

print("✅ Pipeline 정의 완료!")

---

## Cell 5: 파이프라인 컴파일

파이프라인을 YAML 파일로 컴파일합니다. 이 YAML 파일을 Kubeflow UI에 업로드하여 실행합니다.

In [ ]:
# YAML 파일로 컴파일
output_file = 'hello_pipeline.yaml'

compiler.Compiler().compile(
    pipeline_func=hello_pipeline,
    package_path=output_file
)

print("=" * 60)
print("✅ 파이프라인 컴파일 완료!")
print("=" * 60)
print(f"\n생성된 파일: {output_file}")
print(f"\n다음 단계:")
print(f"  1. Kubeflow Dashboard 접속")
print(f"  2. Pipelines → Upload pipeline")
print(f"  3. {output_file} 파일 업로드")
print(f"  4. Create run → 파라미터 설정 → Start")
print(f"\n권장 테스트 파라미터:")
print(f"  - a=10, b=20, factor=3 → 예상 결과: 90")
print(f"  - a=7, b=3, factor=5 → 예상 결과: 50")
print("=" * 60)

---

## Cell 6: 컴파일된 YAML 확인 (선택사항)

컴파일된 YAML 파일의 내용을 확인합니다.

In [ ]:
# YAML 파일 내용 확인
with open(output_file, 'r') as f:
    yaml_content = f.read()
    print("=== YAML 파일 내용 (처음 1000자) ===")
    print(yaml_content[:1000])
    print("\n... (생략) ...")

In [ ]:
# 파일 존재 확인
import os
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file)
    print(f"✅ 파일 확인: {output_file} ({file_size} bytes)")
else:
    print(f"❌ 파일을 찾을 수 없습니다: {output_file}")

---

## Cell 7: Component 로컬 테스트

Kubeflow에 업로드하기 전에 Component 함수를 로컬에서 테스트할 수 있습니다.

**⚠️ 참고:** `component.python_func`으로 컴포넌트의 원본 Python 함수에 접근할 수 있습니다.

In [ ]:
# Component를 일반 함수처럼 호출하여 테스트
print("=== 로컬 테스트 시작 ===")
print()

# Step 1: add 테스트
a, b = 10, 20
sum_result = add.python_func(a, b)
print(f"Step 1 결과: {sum_result}")
print()

# Step 2: multiply 테스트
factor = 3
product_result = multiply.python_func(sum_result, factor)
print(f"Step 2 결과: {product_result}")
print()

# Step 3: print_result 테스트
print_result.python_func(product_result)

print()
print("=== 로컬 테스트 완료 ===")

---

## Cell 8: 다양한 테스트 케이스

다양한 파라미터로 파이프라인 로직을 테스트해봅니다.

In [ ]:
# 테스트 케이스 정의
test_cases = [
    {"a": 3, "b": 5, "factor": 2, "expected": 16},
    {"a": 10, "b": 20, "factor": 3, "expected": 90},
    {"a": 7, "b": 3, "factor": 5, "expected": 50},
    {"a": 100, "b": 200, "factor": 2, "expected": 600},
]

print("=" * 60)
print("테스트 케이스 검증")
print("=" * 60)
print(f"{'a':>5} | {'b':>5} | {'factor':>6} | {'expected':>8} | {'actual':>8} | {'result':>8}")
print("-" * 60)

for tc in test_cases:
    # 파이프라인 로직 실행
    sum_result = add.python_func(tc["a"], tc["b"])
    actual = multiply.python_func(sum_result, tc["factor"])
    
    # 결과 검증
    result = "✅ PASS" if actual == tc["expected"] else "❌ FAIL"
    print(f"{tc['a']:>5} | {tc['b']:>5} | {tc['factor']:>6} | {tc['expected']:>8} | {actual:>8} | {result:>8}")

print("=" * 60)

---

## ✅ 완료 체크리스트

### Part 1: 파이프라인 이해 (10분)
- [ ] Component 개념 이해
- [ ] Pipeline 구조 이해
- [ ] DAG (실행 흐름) 이해

### Part 2: 파이프라인 작성 (15분)
- [ ] Component 3개 정의 완료 (add, multiply, print_result)
- [ ] Pipeline 정의 완료
- [ ] 파이프라인 컴파일 성공
- [ ] hello_pipeline.yaml 파일 생성 확인

### Part 3: 파이프라인 실행 (15분)
- [ ] Kubeflow Dashboard 접속
- [ ] 파이프라인 업로드 성공
- [ ] Run 생성 및 시작
- [ ] Graph 탭에서 DAG 확인
- [ ] 각 Component 로그 확인
- [ ] 최종 결과 확인 (90)

---

## 🎯 학습 성과

이 실습을 완료하면:

1. ✅ **KFP SDK v2** 기본 사용법
2. ✅ **Component 정의** 방법 (`@dsl.component`)
3. ✅ **Pipeline 구성** 방법 (`@dsl.pipeline`)
4. ✅ **파이프라인 컴파일** (Python → YAML)
5. ✅ **Kubeflow UI 활용** (업로드, 실행, 모니터링)
6. ✅ **Component 간 데이터 전달** (`.output` 사용)

---

## 💡 문제 해결

### 문제 1: "ModuleNotFoundError: No module named 'kfp'"
```python
# Cell에서 직접 설치
!pip install --upgrade kfp>=2.0.0
```

### 문제 2: Pipeline 업로드 후 "Upload failed"
- YAML 파일을 재컴파일해보세요.
- 파일이 정상적으로 생성되었는지 확인하세요.

### 문제 3: "UTF-8 Collation Error"
- Pipeline/Component 이름에 한글 사용 금지!
- ❌ "헬로 파이프라인" → ✅ "Hello Pipeline"

---

© 2025 현대오토에버 MLOps Training